# Evidenz zur Klassifikation von Wohnungsbewohnerkategorien (Datensatz "wohnungen_2.csv")
Dieses Notebook stellt ein Modell auf Basis der Evidenztheorie vor, das verwendet wird, um die Bewohnerkategorie einer Wohnung basierend auf den Merkmalen der Wohnung zu klassifizieren.

## Ziel
Das Ziel dieses Projekts ist es, die Zusammenhänge zwischen den Eigenschaften einer Wohnung und den Bewohnerkategorien zu modellieren. Wir verwenden die Evidenztheorie, um Wahrscheinlichkeiten vorherzusagen, auch wenn einige Informationen unvollständig sind.

---

## Vorbereitung
Zuerst werden die benötigten Bibliotheken importiert.

In [8]:
import pandas as pd
from data_cleaning.text_to_numeric import get_zimmer, get_stockwerk, get_kindergarten, get_schule, get_bahn, get_miete, \
    get_nebenkosten, get_alter, get_stadtmitte, get_kaution, get_kueche, get_bad, get_mobliert, get_quadratmeter, \
    get_yes_no
from combined_mass.combined_mass import calculate_combined_mass

## Reinigung des Datensatzes
- **Bewohnerkategorie**: Der Datensatz enthält mehrere Spalten, mit anderen Bewohnergruppen als in `wohnungen_1.csv`. Und Zwar gibt es jetzt die Bewohnerkategorie "SingleHighIncome" anstatt "Alleinerziehende". Diese werden in einer neuen Spalte `Bewohnerkategorie` zusammengefasst.
- **Wohnungsattribute**: Der Datensatz enthält auch neue Attribute wie "Entfernung zur Stadtmitte", die bereinigt werden müssen. Ansonsten gibt es Attribute wie "Heizung" nicht mehr, die in `wohnungen_1.csv` vorhanden waren. Die anderen Attribute wurden jedoch beibehalten.
- **Datenbereinigung**: Um den Datensatz korrekt nutzen zu können, müssen wir den Fließtext durch Zahlen ersetzen, damit die KI mit diesen Daten arbeiten kann. Dafür werden "nein" und "ja" in numerische Werte (0 und 1) umgewandelt. Werte wo es mehr als ein Wert gibt wie bei der Miete, wurde der Mittelwert (gerundet) der beiden Werte genommen. Diese Funktionen sind in `data_cleaning/text_to_numeric.py` zu finden. In dieser Python-Datei wurden weitere Funktionen für diesen Datensatz hinzugefügt, um neu hinzugekommene Attribute wie "Entfernung zur Stadtmitte" zu bereinigen.

In [9]:
file_path = '../../data/Wohnungen_2.csv'
data = pd.read_csv(file_path, sep=';') # Gegebenen Datensatz einlesen

# Bewohnerkategorie bestimmen
bewohnerkategorie = ['Studierende', 'Kleinfamilie', 'DINK', 'SingleHighIncome', 'Expatriate', 'Rentnerpaar']
data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(lambda row: 'Keine' if all(row == 'nein') else ', '.join(row[row == 'ja'].index), axis=1)

# Löschen von Daten, die nicht kategorisiert sind (dazu später mehr)
data = data[data['Bewohnerkategorie'] != 'Keine']

# Spalten, in denen "nein" durch 0 und "ja" durch 1 ersetzt werden sollen
columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

# Ersetze "nein" und "ja" in den angegebenen Spalten
pd.set_option('future.no_silent_downcasting', True)
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1}).infer_objects(copy=False)

# Fließtext durch Ganzzahl ersetzen
data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Entfernung zur Stadtmitte'] = data['Entfernung zur Stadtmitte'].apply(get_stadtmitte)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

print(data.head())

    Zimmerzahl  Stockwerk  Hausmeister  Kindergarten  Schule  S-Bahn  Garage  \
3            0          3            1             2       0       0       0   
4            2          4            0             1       2       0       0   
6            2          1            0             2       2       3       0   
7            2          0            0             2       2       3       1   
16           0          4            0             0       2       2       1   

    Miete  Nebenkosten  Alter  ...  Kehrwoche  Moebliert  Quadratmeter  \
3     276           50     63  ...          0          0            25   
4     626          226     18  ...          0          0            96   
6     626          226      6  ...          0          0            96   
7     726          226      0  ...          0          0           106   
16    276           75     26  ...          1          0            25   

    Studierende  Kleinfamilie  DINK  SingleHighIncome  Expatriate  \
3    

### Modelle erstellen und trainieren
Zuerst wird die Menge Omega mit den zu bestimmenden Attributen, d.h. der Bewohnerkategorie, bestimmt. Hier ist im gegensatz zum ersten Datensatz die Bewohnerkategorie "SingleHighIncome" anstatt "Alleinerziehende" hinzugekommen.

In [10]:
omega = ["Studierende", "Kleinfamilie", "DINK", "SingleHighIncome", "Expatriate", "Rentnerpaar"]

Wie in `evidenztheorie_wohnungen_1.ipynb` wird die Funktion `calculate_combined_mass()` verwendet, um die wahrscheinlichste Bewohnerkategorie zu ermitteln. Unten ist ein Beispiel mit dem neuen Attribut "Entfernung zur Stadtmitte".

In [11]:
# Gegebene Attributkombinationen
attributes_values = {
    'Stockwerk': get_stockwerk("EG"),
    'Entfernung zur Stadtmitte': get_stadtmitte("> 30 km"),
    'Kaution': get_kaution("1000-1500"),
    'Kueche': get_kueche("Kueche (neu)")
}

# Berechnung der kombinierten Evidenz (Unsicherheit ist auch enthalten)
combined_mass = calculate_combined_mass(data, attributes_values, omega)

# Ausgabe der kombinierten Evidenz
print("Kombinierte Evidenz:", combined_mass)

# Wahrscheinlichste Hypothese basierend auf der kombinierten Evidenz
most_likely = combined_mass.max_bel()

# Extrahiere den Namen der Bewohnerklasse aus dem frozenset
if isinstance(most_likely, frozenset):
    most_likely = next(iter(most_likely), None)  # Extrahiere das einzige Element oder None, falls leer

print("\nAm wahrscheinlichsten ist:", most_likely)

Basismaße für Stockwerk=0: {set():0.6621621621621622; {'Kleinfamilie'}:0.21621621621621623; {'Studierende'}:0.08108108108108109; {'DINK'}:0.013513513513513514; {'SingleHighIncome'}:0.013513513513513514; {'Expatriate'}:0.013513513513513514; {'Rentnerpaar'}:0.0}
Basismaße für Entfernung zur Stadtmitte=30: {set():0.8783783783783784; {'Kleinfamilie'}:0.0945945945945946; {'Studierende'}:0.013513513513513514; {'SingleHighIncome'}:0.013513513513513514; {'DINK'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Kleinfamilie'}:0.9411764705882353; {'Studierende'}:0.05042016806722689; {'SingleHighIncome'}:0.008403361344537816; {'DINK'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Kaution=1250: {set():0.9324324324324325; {'Kleinfamilie'}:0.02702702702702703; {'DINK'}:0.02702702702702703; {'SingleHighIncome'}:0.013513513513513514; {'Studierende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Kleinfamilie'}:0.9955555555555556; {'SingleHighIn

Auch hier ist die Evidenz 0 möglich, wenn die Basismaße nicht übereinstimmen. Dies wird im folgenden Beispiel veranschaulicht.

In [13]:
attributes_values = {
    'Aufzug': get_yes_no("ja"),
    'Miete': get_miete("251-300"),
    'Nebenkosten': get_nebenkosten("ueber 300"),
    'Entfernung zur Stadtmitte': get_stadtmitte("Zentrum"),
}

combined_mass = calculate_combined_mass(data, attributes_values, omega)
print("Kombinierte Evidenz:", combined_mass)
most_likely = combined_mass.max_bel()

if isinstance(most_likely, frozenset):
    most_likely = next(iter(most_likely), None)

print("\nAm wahrscheinlichsten ist:", most_likely)

Basismaße für Aufzug=1: {set():0.5540540540540541; {'Kleinfamilie'}:0.20270270270270271; {'DINK'}:0.13513513513513514; {'Studierende'}:0.0945945945945946; {'Expatriate'}:0.013513513513513514; {'SingleHighIncome'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Miete=276: {set():0.8108108108108107; {'Studierende'}:0.1891891891891892; {'Kleinfamilie'}:0.0; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Studierende'}:1.0; {'Kleinfamilie'}:0.0; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Nebenkosten=300: {set():0.9324324324324325; {'DINK'}:0.04054054054054054; {'Kleinfamilie'}:0.02702702702702703; {'Studierende'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Konflikt erkannt: Keine gemeinsame Hypothese zwischen den Massen.
Basismaße für Entfernung zur Stadtmitte=0: {set():0.7837837837837838; {'Studierende'}:0.0945945945945946; {'Kleinfamilie'}:0.0945945945945946; {